# Minimal example

**The problem**:

Running the image processing using `docarray` I found that for some reason empty output is generated and printed to the cell output. I'd like to hide that output if possible, or not produce it in the first place.

**Context**:

I am working on a project where one object has a bunch of associated images. Also the number of images may vary between objects. Hence `docarray` seems like a neat tool to solve the problem. In order to loop those images I am using `docarray` helper methods, e.g. `map` and `map_batch`. I've just started using this package, so the source of my problem is likely my on my side. But any help would be greatly appreciated.

In [ ]:
from docarray import Document, DocumentArray, dataclass
from docarray.typing import Image
from typing import List

For reproducibility let's load one image an re-use it

`wget https://docarray.jina.ai/_images/apple1.png`

In [ ]:
uri = "apple1.png"

Let's use `dataclass` as a template for the objects with multiple images. 

In [ ]:
@dataclass
class Basket: # this is where the apples go
    images: List[Image] 

Generating the baskets

In [ ]:
n_baskets = 10
n_items = 3 # all baskets have the same number of apples for simplicity

baskets = DocumentArray([
    Document(Basket(images= [uri for _ in range(n_items)]))
    for _ in range(n_baskets)
])

To step through our baskets batch-wise let's define our helper functions

In [ ]:
image_size = (64,64)

def img_func(d:Document): # tensor transforms here don't seem to generate the output
    return (d
    .set_image_tensor_shape(image_size)
    .set_image_tensor_normalization()
    .set_image_tensor_channel_axis(original_channel_axis=-1, new_channel_axis=0)
    )

def chunk_func(d:Document):   
    # iterating Basket.images 
    return DocumentArray(d.chunks[0].chunks.map(img_func)) # <= empty output generated here?

def batch_func(b:DocumentArray):
    # iterating iterate Basket documents in batch
    return b.map(chunk_func) # <= empty output generated here?


Actually doing the looping

In [ ]:
batch_size = 5

for i, b in enumerate(baskets.map_batch(batch_func, batch_size=batch_size)):
    images = [_d.tensors for _d in b]
    